In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 1 : PreProcess Data

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator,load_img
import numpy as np
import os
#import Pillow
#from PIL import Image
#import opencv
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array, array_to_img
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, initializers

from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np



img_h, img_w = 400,400
channels = 3
batch_size = 30               
train_dir = '/kaggle/input/breakhis-400x/BreaKHis 400X/train' 
test_dir = '/kaggle/input/breakhis-400x/BreaKHis 400X/test'
result_dir = '/kaggle/output/Kaggle/working'        




In [ ]:
train_normal_files = os.listdir(os.path.join(train_dir,'benign'))
print(len(train_normal_files))
#371

train_malignant_files = os.listdir(os.path.join(train_dir,'malignant'))
print(len(train_malignant_files))
#777

test_normal_files = os.listdir(os.path.join(test_dir,'benign'))
print(len(test_normal_files))
#371

test_malignant_files = os.listdir(os.path.join(test_dir,'malignant'))
print(len(test_malignant_files))
#777

num_train=len(train_normal_files)+len(train_malignant_files)
num_validation=len(test_normal_files)+len(test_malignant_files)

print(num_train)
print(num_validation)



In [ ]:
#Number of Learning: 1148 pictures
#Number of Validation:545 pictures

In [ ]:
import cv2

# !ls -lh '/content/gdrive/My Drive/Colab Notebooks/BreakHis/data/train'

normal_files = os.listdir(os.path.join(train_dir,'benign'))[:12]
print(normal_files)

pneumonia_files = os.listdir(os.path.join(train_dir,'malignant'))[:12]
print(pneumonia_files)

# Step 2 : Imaging Data Visual Check

In [ ]:


import numpy as np
import cv2
from matplotlib import pyplot as plt

%matplotlib inline

fig = plt.figure(figsize=(12,20))
plt.subplots_adjust(wspace=0.5, hspace=0.5)

for i in range(12):
  img = cv2.imread(os.path.join(train_dir,'benign',normal_files[i]))
  ax = fig.add_subplot(6,2,i+1)
  ax.set_title('benign'+'- '+normal_files[i])
  a='('+str(img.shape[0])+','+str(img.shape[1])+')'
  ax.set_xlabel(a)
  ax.imshow(img)
plt.show()

fig = plt.figure(figsize=(12,20))
plt.subplots_adjust(wspace=0.5, hspace=0.5)

for i in range(12):
  img = cv2.imread(os.path.join(train_dir,'malignant',pneumonia_files[i]))
  ax = fig.add_subplot(6,2,i+1)
  ax.set_title('malignant'+'- '+normal_files[i])
  a='('+str(img.shape[0])+','+str(img.shape[1])+')'
  ax.set_xlabel(a)
  ax.imshow(img)
plt.show()

# Step 3 : Model Fitting

# # Load Module 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator,load_img
import numpy as np
import os
#import Pillow
#from PIL import Image
#import opencv
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array, array_to_img
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, initializers

from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
import tensorflow as tf
import keras
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D,Input, GlobalMaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation,GlobalAveragePooling2D,Input
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from keras import optimizers


# # Define Model

In [ ]:
## InceptionResNetV2 model

img_h, img_w = 400,400
channels = 3
batch_size = 30  
nb_classes = 1

#batch_size = 30  
#nb_classes = 1

def analysis_model():
    
    # Load InceptionResNetV2 / Because FC layer is unecessary, set include_top=False
    input_tensor = Input(shape=(img_h,img_w,channels))
    inception_v3 = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor)
    
    # Load InceptionResNetV2 /  frozen the portion of weight of model.
    for layer in inception_v3.layers[:775]:
        layer.trainable = False
    for layer in inception_v3.layers[775:]:
        layer.trainable = True
        
    # define FC Layer
        
    model = Sequential()
    model.add(inception_v3)
    model.add(GlobalMaxPooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid', name='sigmoid'))

   # compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1e-5),
                  metrics=['accuracy'])
    
    return model



# # Learning Rate Set Up

In [ ]:
    
def step_decay(epoch):
    initial_lrate = 0.00001 # initial rate for learning
    drop = 0.5              # decay rate 50%
    epochs_drop = 10.0      # decay per 10 epochs
    lrate = initial_lrate * math.pow(
            drop,
            math.floor((epoch)/epochs_drop)
             )
    return lrate
    
    

# # LRHistory Class

In [ ]:
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, Callback

class LRHistory(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.lr = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.lr.append(step_decay(len(self.acc)))
    

# # Data Augumentation

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.8,
    shear_range=0.2,     
)


train_generator = train_datagen.flow_from_directory(
    train_dir,            
    target_size=(img_h, img_w), 
    batch_size=batch_size,      
    class_mode='binary',        
)


test_datagen = ImageDataGenerator(rescale=1.0 / 255)



validation_generator = test_datagen.flow_from_directory(
    test_dir,       
    target_size=(img_h, img_w), 
    batch_size=batch_size,      
    class_mode='binary',      
)

# # Model Fitting

In [ ]:
    def train(train_generator,validation_generator,num_train,num_validation,epochs):
        
        model=analysis_model()
        lr_history=LRHistory()
        lrate = LearningRateScheduler(step_decay())
        callbacks_list=[lr_history,lrate]
        
        
        history = model.fit(
            train_generator,
            
            epochs=epochs,
            
            validation_data=validation_generator,
            
            validation_steps=num_validation//batch_size,
            
            steps_per_epoch=num_train//batch_size,
            
            verbose=1,
            
            callbacks=callbacks_list)
        
        return history, lr_history

# # Execution of model fitting



In [ ]:
    %%timeit
    
    history, lr_history =train(train_generator,validation_generator,1148,545,10)
    

# Step 4 : Plot for Accuracy, Loss and Learning Rate

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_acc_loss_lr(history):
    # plot for accuracy
    plt.plot(history.history['accuracy'],"-",label="accuracy")
    plt.plot(history.history['val_accuracy'],"-",label="val_acc")
    plt.title('accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()

    # plot for loss
    plt.plot(history.history['loss'],"-",label="loss",)
    plt.plot(history.history['val_loss'],"-",label="val_loss")
    plt.title('loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='upper right')
    plt.show()
    
   # plot for learning rate
    plt.plot(lr_history.lr,label="learning rate",color='blue')
    plt.title('Learning Rate')
    plt.xlabel('epoch')
    plt.ylabel('Learning Rate')
    plt.legend(loc='upper right')
    plt.show()
    
# output 
plot_acc_loss_lr(history)

# Step 5 : Conclusion
